# ELT Result

### Import dependencies

In [52]:
import pandas as pd
from sqlalchemy import create_engine
from config import (ServerName, UserName, Password, port, DataBase)
import psycopg2


def getData(sql):
    engine = create_engine(f'postgres+psycopg2://{UserName}:{Password}@{ServerName}:{port}/{DataBase}')
    df_query = pd.read_sql(sql, engine)
    return df_query

### Database Queries

#### What is the relationship between suicide rates in males and females and:
- the country they live, 
- the contries econimic situation (GDP Growth and Inflation)
- and their human development situation (HDI - Human Development Index)

In [53]:
sql_0 = """----Relationship of suicide rates with econimic and development situation by country
        select "Economic"."Country",
        "Economic"."Year",
        "Economic"."GDP growth" as "GDP Growth",
        "Economic"."Inflation",
        "Suicide"."Suicide_Rate_Female",
        "Suicide"."Suicide_Rate_Male",
        "Hum_Dev_Ind"."human_dev_ind" as "Human Development Index"
        from "Economic"
        join "Suicide"
        on "Economic"."Country" = "Suicide"."Country" and "Economic"."Year" = "Suicide"."Year" 
        join "Hum_Dev_Ind"
        on "Suicide"."Country" = "Hum_Dev_Ind"."Country" and "Suicide"."Year" = "Hum_Dev_Ind"."Year" 
    """
total_relation = getData(sql_0)
total_relation.head()

,Country,Year,GDP Growth,Inflation,Suicide_Rate_Female,Suicide_Rate_Male,Human Development Index
0,Afghanistan,2000,0,0,1.0,10.0,..
1,Afghanistan,2005,11.1752702416587,12.68626872,1.5,10.8,0.408
2,Afghanistan,2010,8.43329048161792,0.892536936,1.4,8.6,0.463
3,Afghanistan,2015,1.31004040372002,-1.533846583,1.5,7.8,0.493
4,Afghanistan,2016,2.36671191565063,2.169452176,1.5,7.6,0.494


#### What are the Top 10 Countries for highest suicide rates for males in 2016?

In [54]:
sql_1 = """--- Top 10 Suicide Rate by Country for Males in 2016
            select "Country","Year","Suicide_Rate_Male" from "Suicide" 
            where "Year"=2016
            order by "Suicide_Rate_Male" desc
            limit 10;
    """
top_suicide_male_2016 = getData(sql_1)
top_suicide_male_2016.head(10)

,Country,Year,Suicide_Rate_Male
0,Lithuania,2016,58.1
1,Russian Federation,2016,55.9
2,Belarus,2016,46.9
3,Guyana,2016,43.7
4,Ukraine,2016,41.1
5,Korea South,2016,38.4
6,Kazakhstan,2016,38.3
7,Latvia,2016,37.6
8,Suriname,2016,34.7
9,Estonia,2016,30.6


#### What are the Top 10 Countries for highest suicide rates for females in 2016?

In [55]:
sql_2 = """--- Top 10 Suicide Rate by Country for Females in 2016
        select "Country","Year","Suicide_Rate_Female" from "Suicide" 
        where "Year"=2016
        order by "Suicide_Rate_Female" desc
        limit 10
    """
top_suicide_female_2016 = getData(sql_2)
top_suicide_female_2016.head(10)

,Country,Year,Suicide_Rate_Female
0,Lesotho,2016,24.4
1,Korea South,2016,15.4
2,India,2016,14.7
3,Guyana,2016,14.4
4,Belgium,2016,13.8
5,Switzerland,2016,12.4
6,France,2016,11.7
7,Japan,2016,11.4
8,Suriname,2016,10.9
9,Sweden,2016,10.5


#### What are the suicide rates for males in countries with the lowest GDP growth in 2016?

In [56]:
sql_3 = """--- Suicide Rate for Males among Countries with lowest GDP growth
        select * 
        from (select s."Country",s."Year",s."Suicide_Rate_Male", e."GDP growth"
        from "Suicide" as s
        join "Economic" as e
        on (s."Country"=e."Country") and (s."Year"=e."Year")) as c
        where c."Year"=2016
        order by c."GDP growth"
        limit 20;
    """
low_gdp_males_2016 = getData(sql_3)
low_gdp_males_2016.head()

,Country,Year,Suicide_Rate_Male,GDP growth
0,Russian Federation,2016,55.9,-0.224910984282204
1,Greece,2016,8.1,-0.24431474084669
2,Belize,2016,7.9,-0.486536511492986
3,Burundi,2016,13.4,-0.570434450658766
4,Angola,2016,7.0,-0.813494095515892


#### What are the suicide rates for males and females in countries with the lowest HDI in 2016?

In [57]:
sql_4 = """--- Suide Rates among Countries with lowest HDI
            select * 
            from (select s."Country",s."Year",s."Suicide_Rate_Male",s."Suicide_Rate_Female", h."human_dev_ind"
            from "Suicide" as s
            join "Hum_Dev_Ind" as h
            on (s."Country"=h."Country") and (s."Year"=h."Year")) as cb
            where cb."Year"=2016
            order by cb."human_dev_ind"
            limit 20;
    """
low_HDI_males_2016 = getData(sql_4)
low_HDI_males_2016.head()

,Country,Year,Suicide_Rate_Male,Suicide_Rate_Female,human_dev_ind
0,Niger,2016,5.9,3.3,0.351
1,Central African Republic,2016,11.5,4.0,0.362
2,South Sudan,2016,4.9,2.6,0.394
3,Chad,2016,10.3,7.4,0.405
4,Sierra Leone,2016,11.3,8.1,0.413


#### What are the global averages for suicides rates for males and females?

In [58]:
sql_5 = """--- Global Suicide Rate for Males and Females
        select "Year",avg("Suicide_Rate_Male") as "Male Avg Suicide Rate",
        avg("Suicide_Rate_Female") as "Female Avg Suicde Rate"
        from "Suicide"
        group by "Year"
        order by "Year";
    """
average_suicide_rate_2016 = getData(sql_5)
average_suicide_rate_2016.head()

,Year,Male Avg Suicide Rate,Female Avg Suicde Rate
0,2000,16.208242,5.798352
1,2005,15.483516,5.519231
2,2010,14.798352,5.154945
3,2015,14.026923,4.959890
4,2016,13.891209,4.869780


#### What are the suicide rates for males and females in countries with the highest inflation in 2016?

In [59]:
sql_6 = """--- Suide Rates among Countries with highest Inflation
            select * 
            from (select s."Country",s."Year",s."Suicide_Rate_Male",s."Suicide_Rate_Female" , e."Inflation"
            from "Suicide" as s
            join "Economic" as e
            on (s."Country"=e."Country") and (s."Year"=e."Year")) as c
            where c."Year"=2016
            order by c."Inflation" desc
            limit 20;
    """
high_inflation_suicide_rate_2016 = getData(sql_6)
high_inflation_suicide_rate_2016.head()

,Country,Year,Suicide_Rate_Male,Suicide_Rate_Female,Inflation
0,Mozambique,2016,7.4,2.5,9.966216216
1,Uruguay,2016,29.2,8.3,9.639413476
2,Liberia,2016,7.1,6.6,8.833967088
3,Nepal,2016,9.7,7.9,8.79034332
4,Brazil,2016,10.0,3.1,8.739478523
